In [1]:
import requests
import json
import csv
import pandas as pd
from matplotlib import pyplot as plt
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC #using alias as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import os

In [2]:
# webscraping function

def show_rooms():
    
    year_to_book = input("enter year ")
    month_to_book = input("please enter the month from Jan-Feb-Mar-Apr-May-Jun-Jul-Aug-Sep-Oct-Nov-Dec ")
    date_to_book = input("please enter the day as number ")
    
    options = Options()
    options.add_argument("start-maximized")

    PATH = Service("/Users/clt-user/Desktop/selenium/chromedriver.exe")
    driver = webdriver.Chrome(service=PATH, options=options)

    # prevents previous data from being used in new trial
    
    if os.path.exists("library.csv"):
        os.remove("library.csv")
    else:
        pass
    
    
    library_slots = open("library.csv", "w", newline="", encoding="utf-8") #creating a csv file to write rows
    csv_writer = csv.writer(library_slots)
    csv_writer.writerow(["Time", "Day", "Date", "Room", "Availability"])


    driver.get("https://www.binghamton.edu/libraries/services/reservations/index.html")

    try:
        #-------------------------Bartle Library----------------------------
        #Click Bartle library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN BARTLE"))
        )

        element.click()
        
        #Click to open drop down menu for bartle library
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']"))
        )

        element.click()


        #Click Show All in the drop down menu
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']/option[5]"))
        )

        element.click()


        #Click Go To Date to open Calendar
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
        )

        element.click()

            #Click datepicker to select month
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
        )

        element.click()

        #click datepicker to select year
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
        )

        element.click()
        
        #selecting year given by user in calendar
        element=   WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
        element.click()

        #Selecting month given by user in calendar
        months = driver.find_elements(By.TAG_NAME, "span")
        for month in months:
            
            if month.text == month_to_book:
                
                month.click()
                
                break

        #selecting date given by user in calender (number date)
        dates = driver.find_elements(By.TAG_NAME, "td")

        flag = False
        for date in dates:
            
            if date.text == "1":
                
                flag = True

            if flag == False:
                
                continue

            if date.text == date_to_book:
                
                date.click()
                
                break

        flag = False
        
        #scraping available slots for the user given date
        if datetime.today().strftime('%Y-%m-%d').split("-")[-1] == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]
            if "Media Viewing Room" not in details:
                room_ = data[5]
                available_ = data[6]            
            else:
                room_ = data[5] + " " + data[6] + " " + data[7] + " " + data[8]
                available_ = data[9]

            csv_writer.writerow([time_, day_, date_, room_, available_])

        driver.back()
        driver.back()

        #-------------------------Science Library----------------------------

        #Click Science library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN THE SCIENCE LIBRARY"))
        )

        element.click()

        #Click Go To Date to open Calendar
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
        )
        
        element.click()
    
        #Click datepicker to select month
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
        )

        element.click()

        #click datepicker to select year
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
        )

        element.click()

         #selecting year given by user in calendar
        element=   WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
        element.click()

        #Selecting month given by user in calendar
        months = driver.find_elements(By.TAG_NAME, "span")
        for month in months:
            
            if month.text == month_to_book:
                
                month.click()
                break

        #selecting date given by user in calender (number date)
        dates = driver.find_elements(By.TAG_NAME, "td")

        flag = False
        for date in dates:

            if date.text == "1":
                
                flag = True
                
            if flag == False:
                
                continue

            if date.text == date_to_book:
                
                date.click()
                break

        flag = False
        
        #scraping available slots for the user given date
        if datetime.today().strftime('%Y-%m-%d').split("-")[-1] == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]
            room_ = data[5] + " " + data[6] + " " + data[7]
            available_ = data[8]
            csv_writer.writerow([time_, day_, date_, room_, available_])


        driver.back()

        #-------------------------UDC Library----------------------------

        #Click UDC library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN THE UDC"))
        )

        element.click()
        
        #Click Go To Date to open Calendar
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
        )

        element.click()
    
        #Click datepicker to select month
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
        )

        element.click()

        #click datepicker to select year
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
        )

        element.click()

        #selecting year given by user in calendar
        element = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
        element.click()

        #Selecting month given by user in calendar
        months = driver.find_elements(By.TAG_NAME, "span")
        for month in months:
            
            if month.text == month_to_book:
                
                month.click()
                break

        #selecting date given by user in calender (number date)
        dates = driver.find_elements(By.TAG_NAME, "td")

        flag = False
        for date in dates:
            
            if date.text == "1":
                
                flag = True

            if flag == False:
                
                continue

            if date.text == date_to_book:
                
                date.click()
                break

        flag = False

        #scraping available slots for the user given date
        if datetime.today().strftime('%Y-%m-%d').split("-")[-1] == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]
            room_ = data[5] + " " + data[6] + " " + data[7]
            available_ = data[8]
            csv_writer.writerow([time_, day_, date_, room_, available_])

        library_slots.close()

    finally:
        driver.quit()

        
    global library_csv
    
    library_csv = pd.read_csv("library.csv") #read csv file which was recently written
    
    print(library_csv)
    
    


# pre-filter function: gives user option to filter or proceed with all rooms that were scraped
    
def filter_rooms():
            
    further = input("Would you like to specify further? (y/n)")

    if further == "y":

        filter_slots()

    else:

        print("Would you like to book one of these rooms, start a new search, or exit?")
        print("1. Book Room")
        print("2. New Search")
        print("3. Exit")
        next_step = int(input("Selection (1-3): "))

        if next_step == 1:

            room_list = library_csv
            room_num = int(input("Please enter the corresponding option number of the room: "))
            book_this_room = room_list.loc[room_num]
            print(book_this_room)
            book_room(book_this_room)

        elif next_step == 2:

            show_rooms()

        elif next_step == 3:
        
            print("Happy studying!")
    
    
    
# filtering function: can filter results from above based on time or location

# one can be done after the other, or the user can choose not to do one or both
    
def filter_slots():

    global library_filter
    
    library_filter = library_csv.copy()
    
    choice = input("Would you like to specify time or location? ")

    # filter by time first:
    
    if choice == "time":

        time_choice = input("Please enter a starting time (H:MMam/pm): ")

        library_filter.drop(library_filter[library_filter["Time"] != time_choice].index, inplace = True)

        library_filter = library_filter.reset_index(drop = True) # new line
        
        print(library_filter)

        further_filter = input("Would you like to now specify by location? (y/n) ")

        if further_filter == "y":

                print("Studying locations are as follows:")
                print("1. Bartle Library")
                print("2. Science Library")
                print("3. University Downtown Center")
                location_choice = int(input("Where would you like to study? (1-3)"))
                
                if location_choice == 1:
                    
                    ln = library_filter[library_filter["Room"].str.contains("LN")]
                    
                    ls = library_filter[library_filter["Room"].str.contains("LS")]
                    
                    bloomberg = library_filter[library_filter["Room"].str.contains("Bloomberg")]
                    
                    room_list = pd.concat([ln, ls, bloomberg], ignore_index = True)
                    
                    print(room_list)
                    
                            
                            
                elif location_choice == 2:
                    
                    blue = library_filter[library_filter["Room"].str.contains("Blue")]
                    
                    grape = library_filter[library_filter["Room"].str.contains("Grape")]
                    
                    grey = library_filter[library_filter["Room"].str.contains("Grey")]
                    
                    orange = library_filter[library_filter["Room"].str.contains("Orange")]
                    
                    room_list = pd.concat([blue, grape, grey, orange], ignore_index = True)
                    
                    print(room_list)
                    
               
                elif location_choice == 3:
                    
                    room_list = library_filter[library_filter["Room"].str.contains("UDC")]
                    
                    print(room_list)
                    
        else:   
            pass
        
        print("Would you like to book one of these rooms, start a new search, or exit?")
        print("1. Book Room")
        print("2. New Search")
        print("3. Exit")
                
        search_or_book = int(input("Selection (1-3): "))
                
                
        if search_or_book == 1:
                    
            room_list = library_filter
            room_num = int(input("Please enter the corresponding option number of the room: "))
            book_this_room = room_list.loc[room_num]
            print(book_this_room)
            book_room(book_this_room)
                    
                    # queue booking function
                    
        elif search_or_book == 2:
                    
            filter_slots()
                    # restart search function
                    
        elif search_or_book == 3:
                    
            print("Goodbye, study hard!")
                    
    # filter by location first:                
                    
    elif choice == "location":
        
        
        print("Studying locations are as follows:")
        print("1. Bartle Library")
        print("2. Science Library")
        print("3. University Downtown Center")
        location_choice = int(input("Where would you like to study? (1-3)"))
        
        if location_choice == 1:
                    
            ln = library_filter[library_filter["Room"].str.contains("LN")]
                    
            ls = library_filter[library_filter["Room"].str.contains("LS")]
                    
            bloomberg = library_filter[library_filter["Room"].str.contains("Bloomberg")]
                    
            room_list = pd.concat([ln, ls, bloomberg], ignore_index = True)
                    
            print(room_list)
                    
                            
                            
        elif location_choice == 2:
                    
            blue = library_filter[library_filter["Room"].str.contains("Blue")]
                    
            grape = library_filter[library_filter["Room"].str.contains("Grape")]
                    
            grey = library_filter[library_filter["Room"].str.contains("Grey")]
                    
            orange = library_filter[library_filter["Room"].str.contains("Orange")]
                    
            room_list = pd.concat([blue, grape, grey, orange], ignore_index = True)
                    
            print(room_list)
            
                    
               
        elif location_choice == 3:
                    
            room_list = library_filter[library_filter["Room"].str.contains("UDC")]
                    
            room_list = room_list.reset_index(drop = True)
            
            print(room_list)
            

        filter_further = input("Would you like to now filter by time? (y/n) ")
        
        if filter_further == "y":
            
            time_choice = input("Please enter a starting time (H:MMam/pm): ")

            room_list.drop(room_list[room_list["Time"] != time_choice].index, inplace = True)

            room_list = room_list.reset_index(drop = True)
            
            print(room_list)
            
        else:
            pass
            
        print("Would you like to book one of these rooms, start a new search, or exit?")
        print("1. Book Room")
        print("2. New Search")
        print("3. Exit")
                
        search_or_book = int(input("Selection (1-3): "))
                
                
        if search_or_book == 1:
                    
            room_num = int(input("Please enter the corresponding option number of the room: "))
            book_this_room = room_list.loc[room_num]
            print(book_this_room)
            book_room(book_this_room)
            
            # queue booking function
                    
        elif search_or_book == 2:
                    
            filter_slots()
                    # restart search function
                    
        elif search_or_book == 3:
            
            print("Goodbye, study hard!")
            
            
        
# pd.concat(): https://www.geeksforgeeks.org/how-to-add-one-row-in-an-existing-pandas-dataframe/#  


# booking function: will automate booking of previously chosen study room


def book_room(book_this_room):

    month_to_book = book_this_room["Date"][0:3]

    n = len(book_this_room["Date"])

    year_to_book = book_this_room["Date"][n-4:n]

    date_to_book = book_this_room["Date"].replace(year_to_book, "")

    time_to_book = book_this_room["Time"]

    room = book_this_room["Room"]
    
    print(month_to_book, year_to_book, date_to_book, time_to_book, room)

    import re

    date_to_book = re.sub('\D', '', date_to_book)

    options = Options()
    options.add_argument("start-maximized")
    
    PATH = Service("/Users/clt-user/Desktop/selenium/chromedriver.exe")
    driver = webdriver.Chrome(service=PATH, options=options)
    
    driver.get("https://www.binghamton.edu/libraries/services/reservations/index.html")
    
    try:
        #-------------------------Bartle Library----------------------------
        #Click Bartle library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN BARTLE"))
        )

        element.click()

        #Click to open drop down menu for bartle library
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']"))
        )

        element.click()


        #Click Show All in the drop down menu
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']/option[5]"))
        )

        element.click()


        today = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        
        if date_to_book == today:
            
            pass
        
        else:
            
            #Click Go To Date to open Calendar
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
            )

            
            element.click()

            #Click datepicker to select month
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
            )

            element.click()

            #click datepicker to select year
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
            )

            element.click()

            #selecting year given by user in calendar
            element= WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
            element.click()

            #Selecting month given by user in calendar
            months = driver.find_elements(By.TAG_NAME, "span")
            for month in months:
                if month.text == month_to_book:
                    month.click()
                    #time.sleep(5)
                    break

            #selecting date given by user in calender (number date)
            dates = driver.find_elements(By.TAG_NAME, "td")

            today = datetime.today().strftime('%Y-%m-%d').split("-")[-1]

            for date in dates:
                if date.text == date_to_book:
                    
                    date.click()
                    #time.sleep(5)
                    break
                    
        #scraping available slots for the user given date
        if datetime.today().strftime('%Y-%m-%d').split("-")[-1] == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]


            if "Media Viewing Room" not in details:
                room_ = data[5]
                available_ = data[6] 


            else:
                room_ = data[5] + " " + data[6] + " " + data[7] + " " + data[8]
                available_ = data[9]


            if room_ == room:

                if time_ == time_to_book:

                    slot.click()

        #times = driver.find_element(By.ID, "bookingend_1")
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(5)
        
        times = WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.ID, "bookingend_1")))
        
        times.click()

        end_times = times.find_elements(By.TAG_NAME, "option")
        
        
        n = 1

        for i in end_times:

            print(f"{n}. {i.text}")

            n += 1

        # display list of all possible times you can have the room until:
        
        print("These are all of the available end times.")

        end_time = int(input(f"Which would you like to choose? (1-{str(n-1)}):"))

        end_time2 = end_times[end_time - 1]

        end_time2.click()

        submit = driver.find_element(By.ID, "submit_times")

        submit.click()

        print("Please log into your Binghamton account to finalize your booking.")
        input("Press enter/return when finished.")
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        time.sleep(5)
        
        finish = WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.ID, "btn-form-submit")))
        finish.click()

        print("Your room has been booked!")
        print("You will shortly recieve a confirmation email. Happy studying!")
        input("To exit, please press enter/return.")

        # add one more option if want to book again or if finished and log out


    finally:
        driver.quit()

In [ ]:
def main():
    
    show_rooms()
    
    filter_rooms()
    
main()

enter year 2023
please enter the month from Jan-Feb-Mar-Apr-May-Jun-Jul-Aug-Sep-Oct-Nov-Dec Apr
please enter the day as number 26
      Time        Day            Date                          Room  \
0  11:00pm  Wednesday  April 26, 2023                      LN-1356E   
1  11:30pm  Wednesday  April 26, 2023                      LN-1356E   
2  11:00pm  Wednesday  April 26, 2023  Media Viewing Room (LS-1546)   
3  11:30pm  Wednesday  April 26, 2023  Media Viewing Room (LS-1546)   
4  11:00pm  Wednesday  April 26, 2023                     Bloomberg   
5  11:30pm  Wednesday  April 26, 2023                     Bloomberg   

  Availability  
0    Available  
1    Available  
2    Available  
3    Available  
4    Available  
5    Available  
